In [1]:
# Import Libs

# General
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read the data
data = pd.read_csv('/kaggle/input/sales-analysis/SalesKaggle3.csv')
data.head()

,Order,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,2,Historical,1737127,0.0,0.0,D,15,1,682743.0,44.99,2015,8,28.97,31.84
1,3,Historical,3255963,0.0,0.0,D,7,1,1016014.0,24.81,2005,39,0.00,15.54
2,4,Historical,612701,0.0,0.0,D,0,0,340464.0,46.00,2013,34,30.19,27.97
3,6,Historical,115883,1.0,1.0,D,4,1,334011.0,100.00,2006,20,133.93,83.15
4,7,Historical,863939,1.0,1.0,D,2,1,1287938.0,121.95,2010,28,4.00,23.99


In [3]:
data['File_Type'].value_counts()

Active        122921
Historical     75996
Name: File_Type, dtype: int64

In [4]:
# Function Defination
def ABC_segmentation(perc):
    """
    Create the 3 classes A, B, C based on
    quantity percentage (A-60%, B-25%, C-15%)
    """
    if perc > 0 and perc <0.6 :
        return 'A'
    elif perc >=0.6 and perc <0.85:
        return 'B'
    elif perc >=0.85:
        return 'C'

In [5]:
# Take a subset of the data, we need to use the price & the quantity of each item
data_sub = data[['SKU_number', 'PriceReg', 'ItemCount', 'File_Type']][(data['File_Type'] == 'Historical')]

data_sub.head()

,SKU_number,PriceReg,ItemCount,File_Type
0,1737127,44.99,8,Historical
1,3255963,24.81,39,Historical
2,612701,46.00,34,Historical
3,115883,100.00,20,Historical
4,863939,121.95,28,Historical


In [6]:
# Create the column of the additive cost per SKU
data_sub['AddCost'] = data_sub['PriceReg'] * data_sub['ItemCount']

# Order By Cumulative Cost
data_sub = data_sub.sort_values(by=['AddCost'], ascending= False)

# Create the column of the running CumCost of the cumulative cost per SKU
data_sub['RunCumCost'] = data_sub['AddCost'].cumsum()

# Create the column of the total sum
data_sub['TotSum'] = data_sub['AddCost'].sum()

# Create the column of the running percentage
data_sub['RunPerc'] = data_sub['RunCumCost']/data_sub['TotSum']

# Create the column of the class
data_sub['Class'] = data_sub['RunPerc'].apply(ABC_segmentation)

In [7]:
# Check the data
data_sub.head()

,SKU_number,PriceReg,ItemCount,File_Type,AddCost,RunCumCost,TotSum,RunPerc,Class
685,145889,244.6,851,Historical,208154.6,208154.6,3.426320e+08,0.000608,A
601,435034,281.8,616,Historical,173588.8,381743.4,3.426320e+08,0.001114,A
75056,538479,2645.3,49,Historical,129619.7,511363.1,3.426320e+08,0.001492,A
5752,212633,235.6,521,Historical,122747.6,634110.7,3.426320e+08,0.001851,A
3758,212480,208.8,579,Historical,120895.2,755005.9,3.426320e+08,0.002204,A


In [8]:
# Total SKUs for each class
data_sub['Class'].value_counts()

C    37413
B    22288
A    16295
Name: Class, dtype: int64

In [9]:
# Total cost per class
print('Cost of Class A:', data_sub[data_sub['Class']=='A']['AddCost'].sum())
print('Cost of Class B:', data_sub[data_sub['Class']=='B']['AddCost'].sum())
print('Cost of Class C:', data_sub[data_sub['Class']=='C']['AddCost'].sum())

Cost of Class A: 205577451.4
Cost of Class B: 85658215.25999999
Cost of Class C: 51396361.129999995


In [10]:
# Percentage of total cost per class
print('Percentage of Cost of Class A:', data_sub[data_sub['Class']=='A']['AddCost'].sum()/data_sub['AddCost'].sum())
print('Percentage of Cost of Class B:', data_sub[data_sub['Class']=='B']['AddCost'].sum()/data_sub['AddCost'].sum())
print('Percentage of Cost of Class C:', data_sub[data_sub['Class']=='C']['AddCost'].sum()/data_sub['AddCost'].sum())

Percentage of Cost of Class A: 0.5999948479013728
Percentage of Cost of Class B: 0.2500006079773141
Percentage of Cost of Class C: 0.15000454412131298
